In [1]:
from lib.imports import *
from lib.functions import *

# square stress

$$
L_{stress} =
\frac{k}{2}
\sum_{(u,v) \in E^*}
\big|
    d(u,v) - \|p_u-p_v\|_2
\big|^2
$$

In [2]:
class StressLoss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, node_pos, batch):
        start, end = get_full_edges(node_pos, batch)
        dist = (start - end).norm(dim=1)
        l, k = batch.edge_attr[:, 0], batch.edge_attr[:, 1]
        return dist.sub(l).square().mul(k/2).sum().div(batch.num_graphs)

$$
L_{normalized\_stress}=
\sum_{(u,v)\in E^*}
\frac{
    \big|
        d(u,v) - \|p_u - p_v\|_2
    \big|^2
}{
    d(u,v)^2
}
$$

$$
L_{normalized\_stress}=
\sum_{(u,v)\in E^*}
\frac{
    \big|
        d(u,v) - \|p_u - p_v\|_2
    \big|^2
}{
    \max
    \big(\big\{
        d(u,v),
        \|p^*_u - p^*_v\|_2
    \big\}\big)^2
},
\text{ where $p^*$ denotes constant values w.r.t. $\nabla L$}
$$

In [2]:
class NormalizedStressLoss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, node_pos, batch):
        start, end = get_full_edges(node_pos, batch)
        dist = (start - end).norm(dim=1)
        l, k = batch.edge_attr[:, 0], batch.edge_attr[:, 1]
        denom = dist * (dist >= l) + l * (dist < l)
        return dist.sub(l).abs().div(denom.detach()).square().sum().div(batch.num_graphs)

# ratio stress

$$
L_{ratio\_stress} =
\sum_{(u,v) \in E^*}
1 - \frac{
    \min\big(\big\{
        d(u,v), 
        \|p_u - p_v\|_2
    \big\}\big)
}{
    \max\big(\big\{
        d(u,v), 
        \|p_u - p_v\|_2
    \big\}\big)
}
$$

In [3]:
class RatioStressLoss(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, node_pos, batch):
        start, end = get_full_edges(node_pos, batch)
        dist = (start - end).norm(dim=1)
        l = batch.edge_attr[:, 0]
        ratio = l.div(dist) * (dist >= l) + dist.div(l) * (dist < l)
        return (1 - ratio).sum().div(batch.num_graphs)

# exp ring

$$
L_{normalized\_exp\_ring} =
\sum_{(u,v) \in E^*}
e^{-\gamma\|p_u - p_v\|_2}
$$

In [4]:
class ExponentialRingLoss(nn.Module):
    def __init__(self, gamma=1):
        super().__init__()
        self.gamma = gamma
        
    def forward(self, node_pos, batch):
        start, end = get_full_edges(node_pos, batch)
        num_nodes = map_node_indices_to_graph_property(batch, batch.edge_index.T[:,0], lambda g: g.num_nodes)
        
        return (end.sub(start)
                   .norm(dim=1)
                   .mul(-self.gamma)
                   .exp()
                   .sum()
                   .div(batch.num_graphs))

# std edge loss

$$
L_{std\_edge} = 
\sqrt{
    \frac{1}{|E|}
    \sum_{l \in E} \frac{|l - l_\mu|^2}{l_\mu^2}
}
$$

$$
L_{rescaled\_std\_edge} = 
|E^*|
\sqrt{
    \frac{1}{|E|}
    \sum_{l \in E} \frac{|l - l_\mu|^2}{l_\mu^2}
}
$$

# var edge loss

$$
L_{var\_edge} = 
\frac{1}{|E|}
\sum_{l \in E} \frac{|l - l_\mu|^2}{l_\mu^2}
$$

$$
L_{rescaled\_var\_edge} = 
\frac{|E^*|}{|E|}
\sum_{l \in E} \frac{|l - l_\mu|^2}{l_\mu^2}
$$

In [ ]:
class RescaledEdgeLengthVarianceLoss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, node_pos, batch):
        edge_idx = get_real_edge_index(batch)
        start, end = get_real_edges(node_pos, batch)
        dist = (start - end).norm(dim=1)
        padded_dist, num_edges = convert_to_padded_batch(dist, batch, node_index=edge_idx[:, 0], return_lengths=True)
        mean_dist = padded_dist.sum(dim=1).div(num_edges)[batch.batch][edge_idx[:, 0]]
        node_counts = map_node_indices_to_graph_property(batch, edge_idx[:, 0], lambda g: g.num_nodes)
        edge_counts = map_node_indices_to_graph_property(batch, edge_idx[:, 0], lambda g: len(get_real_edge_index(g)))
        
        return (dist.sub(mean_dist)
                    .div(mean_dist)
                    .square()
                    .div(edge_counts)
                    .mul(node_counts.square().sub(node_counts))
                    .sum()
                    .div(batch.num_graphs))

# sine angle

$$
L_{sine\_angle} = 
\sum_{u \in V}
\sum_{\theta_u^{(i)} \in \text{angles}(u)} 
\sin \bigg( \frac{1}{2} \bigg|
    \frac{2\pi}{\deg(u)} - \theta_u^{(i)}
\bigg| \bigg)
$$

In [5]:
class SineAngularLoss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, node_pos, batch):
        theta, degrees = get_radians(node_pos, batch, return_node_degrees=True)
        phi = degrees.float().pow(-1).mul(2*np.pi)
        return phi.sub(theta).abs().div(2).sin().sum().div(batch.num_graphs)

$$
L_{rescaled\_sine\_angle} = 
\frac{|E^*|}{|V|}
\sum_{u \in V}
\sum_{\theta_u^{(i)} \in \text{angles}(u)} 
\sin \bigg( \frac{1}{2} \bigg|
    \frac{2\pi}{\deg(u)} - \theta_u^{(i)}
\bigg| \bigg)
$$

In [6]:
class RescaledSineAngularLoss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, node_pos, batch):
        theta, degrees, num_nodes = get_radians(node_pos, batch, return_node_degrees=True, return_num_nodes=True)
        phi = degrees.float().pow(-1).mul(2*np.pi)
        return phi.sub(theta).abs().div(2).sin().mul(num_nodes-1).sum().div(batch.num_graphs)

$$
L_{rescaled\_normalized\_sine\_angle} = 
\frac{|E^*|}{|V|}
\sum_{u \in V}
\frac{
    1
}{
    \deg(u)
    \sin\big(
        \frac{\pi}{\deg(u)}
    \big)
}
\sum_{\theta_u^{(i)} \in \text{angles}(u)} 
\sin \bigg( \frac{1}{2} \bigg|
    \frac{2\pi}{\deg(u)} - \theta_u^{(i)}
\bigg| \bigg)
$$

In [7]:
class NormalizedSineAngularLoss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, node_pos, batch):
        theta, node_degrees, num_nodes = get_radians(node_pos, batch, return_node_degrees=True, return_num_nodes=True)
        phi = node_degrees.float().pow(-1).mul(2*np.pi)
        return phi.sub(theta).div(2).abs().sin().div(node_degrees).div(phi.div(2).sin()).mul(num_nodes-1).sum().div(batch.num_graphs)

$$
L_{rescaled\_reweighted\_sine\_angle} = 
\frac{|E^*|}{2|E|}
\sum_{u \in V}
\frac{
    \deg(u)
}{
    \deg(u)
    \sin\big(
        \frac{\pi}{\deg(u)}
    \big)
}
\sum_{\theta_u^{(i)} \in \text{angles}(u)} 
\sin \bigg( \frac{1}{2} \bigg|
    \frac{2\pi}{\deg(u)} - \theta_u^{(i)}
\bigg| \bigg)
$$

# L1 angle

$$
L_{l1\_angle}=\sum_{u\in V}\sum_{\theta_u^{(i)}\in \text{angles}(u)} \bigg|\frac{2\pi}{\deg(u)}-\theta_u^{(i)}\bigg|
$$

In [8]:
class L1AngularLoss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, node_pos, batch):
        theta, degrees = get_radians(node_pos, batch, return_node_degrees=True)
        phi = degrees.float().pow(-1).mul(2*np.pi)
        return phi.sub(theta).abs().sum().div(batch.num_graphs)

$$
L_{rescaled\_l1\_angle} =
\frac{|E^*|}{|V|}
\sum_{u \in V}
\sum_{\theta_u^{(i)} \in \text{angles}(u)}
\bigg|
    \frac{2\pi}{\deg(u)} - \theta_u^{(i)}
\bigg|
$$

In [9]:
class RescaledL1AngularLoss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, node_pos, batch):
        theta, degrees, num_nodes = get_radians(node_pos, batch, return_node_degrees=True, return_num_nodes=True)
        phi = degrees.float().pow(-1).mul(2*np.pi)
        return phi.sub(theta).abs().mul(num_nodes-1).sum().div(batch.num_graphs)

$$
L_{rescaled\_normalized\_l1\_angle} =
\frac{|E^*|}{|V|}
\sum_{u \in V}
\frac{
    \text{deg}(u)
}{
    4\pi\big(
        \text{deg}(u) - 1
    \big)
}
\sum_{\theta_u^{(i)} \in \text{angles}(u)}
\bigg|
    \frac{2\pi}{\deg(u)} - \theta_u^{(i)}
\bigg|
$$

In [10]:
class NormalizedL1AngularLoss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, node_pos, batch):
        theta, degrees, num_nodes = get_radians(node_pos, batch, return_node_degrees=True, return_num_nodes=True)
        phi = degrees.float().pow(-1).mul(2*np.pi)
        return phi.sub(theta).abs().mul(degrees).div(degrees-1).div(4*np.pi).mul(num_nodes-1).sum().div(batch.num_graphs)

$$
L_{rescaled\_reweighted\_l1\_angle} =
\frac{|E^*|}{2|E|}
\sum_{u \in V}
\frac{
    \text{deg}(u)^2
}{
    4\pi\big(
        \text{deg}(u) - 1
    \big)
}
\sum_{\theta_u^{(i)} \in \text{angles}(u)}
\bigg|
    \frac{2\pi}{\deg(u)} - \theta_u^{(i)}
\bigg|
$$

# L2 angle

$$
L_{l2\_angle}=\sqrt{\sum_{u\in V}\sum_{\theta_u^{(i)}\in \text{angles}(u)} \bigg|\frac{2\pi}{\deg(u)}-\theta_u^{(i)}\bigg|^2}
$$

In [11]:
class L2AngularLoss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, node_pos, batch): 
        theta, degrees = get_radians(node_pos, batch, return_node_degrees=True)
        phi = degrees.float().pow(-1).mul(2*np.pi)
        return phi.sub(theta).square().sum().div(batch.num_graphs).abs().sqrt()

$$
L_{rescaled\_l2\_angle} =
\frac{|E^*|}{|V|}
\sqrt{
    \frac{1}{|V|}
    \sum_{u \in V}
    \sum_{\theta_u^{(i)} \in \text{angles}(u)}
    \bigg|
        \frac{2\pi}{\deg(u)} - \theta_u^{(i)}
    \bigg|^2
}
$$

In [12]:
class RescaledL2AngularLoss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, node_pos, batch):
        theta, degrees, num_nodes = get_radians(node_pos, batch, return_node_degrees=True, return_num_nodes=True)
        phi = degrees.float().pow(-1).mul(2*np.pi)
        return phi.sub(theta).square().mul(num_nodes.sub(1).square().mul(num_nodes)).sum().div(batch.num_graphs).abs().sqrt()

$$
L_{rescaled\_normalized\_l2\_angle} =
\frac{|E^*|}{|V|}
\sqrt{
    \frac{1}{|V|}
    \sum_{u \in V}
    \frac{
        \text{deg}(u)
    }{
        4\pi^2\big(
            \text{deg}(u) - 1
        \big)
    }
    \sum_{\theta_u^{(i)} \in \text{angles}(u)} 
    \bigg|
        \frac{2\pi}{\deg(u)} - \theta_u^{(i)}
    \bigg|^2
}
$$

In [13]:
class NormalizedL2AngularLoss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, node_pos, batch):
        theta, degrees, num_nodes = get_radians(node_pos, batch, return_node_degrees=True, return_num_nodes=True)
        phi = degrees.float().pow(-1).mul(2 * np.pi)
        return phi.sub(theta).square().mul(degrees).div(degrees-1).div(4*np.pi**2).mul(num_nodes.sub(1).square().mul(num_nodes)).sum().div(batch.num_graphs).abs().sqrt()

$$
L_{rescaled\_reweighted\_l2\_angle} =
\frac{|E^*|}{|V|}
\sqrt{
    \frac{1}{2|E|}
    \sum_{u \in V}
    \frac{
        \text{deg}(u)^2
    }{
        4\pi^2\big(
            \text{deg}(u) - 1
        \big)
    }
    \sum_{\theta_u^{(i)} \in \text{angles}(u)} 
    \bigg|
        \frac{2\pi}{\deg(u)} - \theta_u^{(i)}
    \bigg|^2
}
$$

# square angle

$$
L_{square\_angle} =
\sum_{u \in V}
\sum_{\theta_u^{(i)} \in \text{angles}(u)}
\bigg|
    \frac{2\pi}{\deg(u)} - \theta_u^{(i)}
\bigg|^2
$$

In [14]:
class SquareAngularLoss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, node_pos, batch):
        theta, degrees = get_radians(node_pos, batch, return_node_degrees=True)
        phi = degrees.float().pow(-1).mul(2*np.pi)
        return phi.sub(theta).square().sum().div(batch.num_graphs)

$$
L_{rescaled\_square\_angle} = 
\frac{|E^*|}{|V|}
\sum_{u \in V}
\sum_{\theta_u^{(i)}\in \text{angles}(u)}
\bigg|\frac{2\pi}{\deg(u)}-\theta_u^{(i)}\bigg|^2
$$

In [15]:
class RescaledSquareAngularLoss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, node_pos, batch):
        theta, degrees, num_nodes = get_radians(node_pos, batch, return_node_degrees=True, return_num_nodes=True)
        phi = degrees.float().pow(-1).mul(2*np.pi)
        return phi.sub(theta).square().mul(num_nodes-1).sum().div(batch.num_graphs)

$$
L_{rescaled\_normalized\_l2\_angle} =
\frac{|E^*|}{|V|}
\sum_{u \in V}
\frac{
    \text{deg}(u)
}{
    4\pi^2\big(
        \text{deg}(u) - 1
    \big)
}
\sum_{\theta_u^{(i)} \in \text{angles}(u)}
\bigg|
    \frac{2\pi}{\deg(u)} - \theta_u^{(i)}
\bigg|^2
$$

In [16]:
class NormalizedSquareAngularLoss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, node_pos, batch):
        theta, degrees, num_nodes = get_radians(node_pos, batch, return_node_degrees=True, return_num_nodes=True)
        phi = degrees.float().pow(-1).mul(2*np.pi)
        return phi.sub(theta).square().mul(degrees).div(degrees-1).div(4*np.pi**2).div(num_nodes).sum().div(batch.num_graphs)

$$
L_{rescaled\_reweighted\_l2\_angle} =
\frac{|E^*|}{2|E|}
\sum_{u \in V}
\frac{
    \text{deg}(u)^2
}{
    4\pi^2\big(
        \text{deg}(u) - 1
    \big)
}
\sum_{\theta_u^{(i)} \in \text{angles}(u)}
\bigg|
    \frac{2\pi}{\deg(u)} - \theta_u^{(i)}
\bigg|^2
$$

# ratio angle

$$
L_{ratio\_angle}=\sum_{u\in V}\sum_{\theta_u^{(i)}\in \text{angles}(u)}1-\frac{\min\big(\big\{\frac{2\pi}{\deg(u)}, \theta_u^{(i)}\big\}\big)}{\max\big(\big\{\frac{2\pi}{\deg(u)}, \theta_u^{(i)}\big\}\big)}
$$

In [17]:
class RatioAngularLoss(nn.Module):
    def __init__(self, eps=1e-5):
        super().__init__()
        self.eps = eps
        
    def forward(self, node_pos, batch):
        theta, degrees = get_radians(node_pos, batch, return_node_degrees=True)
        phi = degrees.float().pow(-1).mul(2*np.pi)
        ratio = theta.div(phi + self.eps) * (phi >= theta) + phi.div(theta + self.eps) * (phi < theta)
        return (1 - ratio).sum().div(batch.num_graphs)

$$
L_{rescaled\_ratio\_angle} =
\frac{|E^*|}{2|E|}
\sum_{u \in V}
\sum_{\theta_u^{(i)} \in \text{angles}(u)}
1 - \frac{
    \min\big(\big\{
        \frac{2\pi}{\deg(u)}, 
        \theta_u^{(i)}
    \big\}\big)
}{
    \max\big(\big\{
        \frac{2\pi}{\deg(u)}, 
        \theta_u^{(i)}
    \big\}\big)
}
$$

In [18]:
class RescaledRatioAngularLoss(nn.Module):
    def __init__(self, eps=1e-5):
        super().__init__()
        self.eps = eps
        
    def forward(self, node_pos, batch):
        theta, degrees, num_nodes, num_real_edges = get_radians(node_pos, batch, 
                                                                return_node_degrees=True, 
                                                                return_num_nodes=True,
                                                                return_num_real_edges=True)
        phi = degrees.float().pow(-1).mul(2*np.pi)
        ratio = theta.div(phi + self.eps) * (phi >= theta) + phi.div(theta + self.eps) * (phi < theta)
        return (1 - ratio).div(num_real_edges).mul(num_nodes.square()-num_nodes).sum().div(batch.num_graphs)

# square ratio angle

$$
L_{rescaled\_square\_ratio\_angle} =
\frac{|E^*|}{2|E|}
\sum_{u \in V}
\sum_{\theta_u^{(i)} \in \text{angles}(u)}\Bigg(
1 - \frac{
    \min\big(\big\{
        \frac{2\pi}{\deg(u)}, 
        \theta_u^{(i)}
    \big\}\big)
}{
    \max\big(\big\{
        \frac{2\pi}{\deg(u)}, 
        \theta_u^{(i)}
    \big\}\big)
}\Bigg)^2
$$